<a href="https://colab.research.google.com/github/ishaanpaul98/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports #

In [75]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import datetime
import yfinance as yf
import time
import requests
import io

In [76]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [78]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from sklearn.metrics import plot_confusion_matrix

# Stock Data Helper Functions #

In [79]:
def getStockDataDaily(symbol, day):
    print("Getting stock data for stock $"+symbol)
    df = yf.download(symbol, start=day, period = "1d")
    return df

getStockDataDaily('AAPL', "2022-12-21")

Getting stock data for stock $AAPL
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-21,132.979996,136.809998,132.750000,135.449997,135.449997,85928000
2022-12-22,134.350006,134.559998,130.300003,132.229996,132.229996,77852100
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.860001,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,130.029999,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,126.040001,85438400
2022-12-29,127.989998,130.479996,127.730003,129.610001,129.610001,75703700
2022-12-30,128.410004,129.949997,127.430000,129.929993,129.929993,76960600
2023-01-03,130.279999,130.899994,124.169998,125.070000,125.070000,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,126.360001,89113600


In [80]:
def arrayToString(arr):
    print("Starting array to list")
    listToStr = ' '.join([str(elem) for elem in arr])
    return listToStr


In [81]:
def getMultiStockDataDaily(symbols, day):
    if len(symbols) > 1:
        print("Length of symbols array is more than 1. STARTING ARRAYTOSTRING")
        symbols = arrayToString(symbols)
    print("Getting stock data for stock $"+symbols)
    df = yf.download(symbols, start=day, period = "1d", group_by='ticker')
    return df

getMultiStockDataDaily(['AAPL', 'TSLA'], "2022-12-21")

Length of symbols array is more than 1. STARTING ARRAYTOSTRING
Starting array to list
Getting stock data for stock $AAPL TSLA
[*********************100%***********************]  2 of 2 completed


TSLA                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2022-12-21  139.339996  141.259995  135.889999  137.570007  137.570007   
2022-12-22  136.000000  136.630005  122.260002  125.349998  125.349998   
2022-12-23  126.370003  128.619995  121.019997  123.150002  123.150002   
2022-12-27  117.500000  119.669998  108.760002  109.099998  109.099998   
2022-12-28  110.349998  116.269997  108.239998  112.709999  112.709999   
2022-12-29  120.389999  123.570000  117.500000  121.820000  121.820000   
2022-12-30  119.949997  124.480003  119.750000  123.180000  123.180000   
2023-01-03  118.470001  118.800003  104.639999  108.099998  108.099998   
2023-01-04  109.110001  114.589996  107.519997  113.639999  113.639999   
2023-01-05  110.510002  111.750000  107.160004  110.339996  110.339996   
2023-01-06  103.000000  114.389999  101.809998  113.059998  113.059998   
2023-01-09  118.959999  123.519997  117.110001  119.769997  119.769997   
2023-01-10  121.070000  122.760002  114.919998  118.849998  118.849998   
2023-01-11  122.089996  125.949997  120.510002  123.220001  123.220001   
2023-01-12  122.559998  124.129997  117.000000  123.559998  123.559998   
2023-01-13  116.550003  122.629997  115.599998  122.400002  122.400002   
2023-01-17  125.699997  131.699997  125.019997  131.490005  131.490005   
2023-01-18  136.559998  136.679993  127.010002  128.779999  128.779999   
2023-01-19  127.260002  129.990005  124.309998  127.169998  127.169998   
2023-01-20  128.679993  133.509995  127.349998  133.419998  133.419998   
2023-01-23  135.869995  145.380005  134.270004  143.750000  143.750000   
2023-01-24  143.000000  146.500000  141.100006  143.889999  143.889999   
2023-01-25  141.910004  146.410004  138.070007  144.429993  144.429993   
2023-01-26  159.970001  161.419998  154.759995  160.270004  160.270004   
2023-01-27  162.429993  180.679993  161.169998  177.899994  177.899994   
2023-01-30  178.050003  179.770004  166.500000  166.660004  166.660004   
2023-01-31  164.570007  174.300003  162.779999  173.220001  173.220001   
2023-02-01  173.889999  183.809998  169.929993  181.410004  181.410004   
2023-02-02  187.330002  196.750000  182.610001  188.270004  188.270004   
2023-02-03  183.949997  199.000000  183.690002  189.979996  189.979996   
2023-02-06  193.009995  198.160004  189.919998  194.759995  194.759995   

                             AAPL                                      \
               Volume        Open        High         Low       Close   
Date                                                                    
2022-12-21  145417400  132.979996  136.809998  132.750000  135.449997   
2022-12-22  210090300  134.350006  134.559998  130.300003  132.229996   
2022-12-23  166989700  130.919998  132.419998  129.639999  131.860001   
2022-12-27  208643400  131.380005  131.410004  128.720001  130.029999   
2022-12-28  221070500  129.669998  131.029999  125.870003  126.040001   
2022-12-29  221923300  127.989998  130.479996  127.730003  129.610001   
2022-12-30  157304500  128.410004  129.949997  127.430000  129.929993   
2023-01-03  231402800  130.279999  130.899994  124.169998  125.070000   
2023-01-04  180389000  126.889999  128.660004  125.080002  126.360001   
2023-01-05  157986300  127.129997  127.769997  124.760002  125.019997   
2023-01-06  220575900  126.010002  130.289993  124.889999  129.619995   
2023-01-09  190284000  130.470001  133.410004  129.889999  130.149994   
2023-01-10  167642500  130.259995  131.259995  128.119995  130.729996   
2023-01-11  183810800  131.250000  133.509995  130.460007  133.490005   
2023-01-12  169400900  133.880005  134.259995  131.440002  133.410004   
2023-01-13  180439300  132.029999  134.919998  131.660004  134.759995   
2023-01-17  186477000  134.830002  137.289993  134.130005  135.940002   
2023-01-18  195680300  136.820007  138.61

In [82]:
def getMonthlyStockData(symbol, day = datetime.date.today() - datetime.timedelta(days = 1), interval = '1mo'):
    print("Getting stock data for stock $"+symbol)
    df = yf.download(symbol, start=day, period = interval, group_by='ticker')
    return df

#getMonthlyStockData('AAPL', "2022-11-21", '1mo')

# Gathering FinViz Data (Today's News) #

In [83]:
# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL']

In [84]:
from finvizfinance.news import News
fnews = News()
all_news = fnews.get_news()
all_news

{'news':        Date                                              Title  \
 0   07:26PM  Adani Group still eligible for CEMBI, JACI, JE...   
 1   07:21PM  Disney's Reedy Creek Improvement District to g...   
 2   07:05PM   Stocks moving after hours: Chegg, Blizzard, more   
 3   07:04PM  One in five homes have not cashed energy vouchers   
 4   07:03PM  Stocks End Lower as Investors Ponder Fed Rate ...   
 ..      ...                                                ...   
 85  07:49AM  Stocks making the biggest moves premarket: Tys...   
 86  07:47AM  Tyson Foods misses profit estimates as lower b...   
 87  07:37AM     Stocks Poised for Losses, Treasury Yields Rise   
 88  07:27AM  India Government Plans to Oppose Vedanta Zinc ...   
 89  07:24AM  Former UK PM Liz Truss is blaming the left-win...   
 
                Source                                               Link  
 0     www.reuters.com  https://www.reuters.com/business/adani-group-s...  
 1     foxbusiness.com  https://fo

In [85]:
# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    print("current url is: " +url)
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36."}
    req = Request(url=url,headers=header) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

current url is: https://finviz.com/quote.ashx?t=AAPL


Recent News Headlines for AAPL: 
Why Taiwan Semiconductor Manufacturing Is Falling Today ( Feb-06-23 11:14AM )
Stock market today: Dow slips as Treasury yields jump to dent tech; Powell eyed ( 11:04AM )
Apple iPhone 14 being sold at discount in China: Report ( 11:00AM )


In [94]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        #print(x.get_text())
        text = x.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text ])
print(parsed_news)

[['AAPL', 'Feb-06-23', '11:14AM', 'Feb-06-23 11:14AMWhy Taiwan Semiconductor Manufacturing Is Falling Today Motley Fool'], ['AAPL', 'Feb-06-23', '11:04AM', '11:04AMStock market today: Dow slips as Treasury yields jump to dent tech; Powell eyed Investing.com'], ['AAPL', 'Feb-06-23', '11:00AM', '11:00AMApple iPhone 14 being sold at discount in China: Report Yahoo Finance Video'], ['AAPL', 'Feb-06-23', '10:57AM', '10:57AM1 Supercharged Nasdaq Stock to Buy Hand Over Fist Before It Jumps Higher Motley Fool'], ['AAPL', 'Feb-06-23', '10:48AM', '10:48AMS&P 500 off lows but remains pressured as Fed jitters drive up Treasury yields Investing.com'], ['AAPL', 'Feb-06-23', '10:00AM', '\n10:00AM\nLoading…\n'], ['AAPL', 'Feb-06-23', '10:00AM', '10:00AMApple Heads the Management Top 250 All-Stars The Wall Street Journal'], ['AAPL', 'Feb-06-23', '09:50AM', '09:50AMStocks open lower, Tesla stock climbs as Apple and Microsoft slide Yahoo Finance Video'], ['AAPL', 'Feb-06-23', '09:33AM', '09:33AMApples La

# Gathering Data From AlphaAdvantage for Historical News #

In [87]:
from decouple import config
import requests
import urllib.parse
import json
import datetime
AAapikey = config('AAKey')

In [88]:
#Function to convert user provided date to date required by AlphaAdvantage
def toAADate(oldDate):
    newDate = oldDate.strftime("%Y%m%dT0130")
    return str(newDate)


In [97]:
#End Date is yesterday
def breakdownofhistoric(ticker, endDate = datetime.date.today() - datetime.timedelta(days = 1), interval = 30, runs = 1):
    print("Starting the breakdown")
    columns = ['parsed_news']
    # Format for parsed_news is [Ticker, Date, Time, Headlines]
    historic_parsed_news = []
    window = 5
    startDate = endDate - datetime.timedelta(days = interval)
    rollingEndDate = startDate + datetime.timedelta(days = window)
    while rollingEndDate <= endDate:
        print("Current Start Date is: "+str(startDate))
        print("Current End Date is: "+str(rollingEndDate))
        startDate = endDate + datetime.timedelta(days = window)
        rollingEndDate = startDate + datetime.timedelta(days = window)
    print(historic_parsed_news)
    return historic_parsed_news


breakdownofhistoric('AAPL', interval = 30)

Starting the breakdown
Current Start Date is: 2023-01-06
Current End Date is: 2023-01-11
[]


[]

In [90]:
# Get data from AlphaAdvantage for one ticker for a particular day
def getHistoricNewsData(ticker, endDate = datetime.date.today() - datetime.timedelta(days = 1) , interval = '1mo'):
    url = 'https://www.alphavantage.co/query?'
    print("endDate is: " +str(endDate))
    if interval == '1mo':
        days_to_add = 30
    else:
        days_to_add = 60
    delta_days = datetime.timedelta(days = days_to_add)
    startDate = endDate - delta_days
    print("startDate is: " +str(startDate))
    if startDate and endDate:
        startDate = toAADate(startDate)
        endDate = toAADate(endDate)
        print("startDate is: " +str(startDate))
        print("endDate is: " +str(endDate))
        Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'time_from': startDate, 'time_to': endDate, 'sort': 'EARLIEST','limit': 200, 'apikey': AAapikey}
    else:
        print("NEED DATES")
        #Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'sort': 'LATEST','limit': 100, 'apikey': AAapikey}
    r = requests.get(url, params = Myparams)
    data = r.json()
    #return data
    historic_news = pd.DataFrame(columns=['Date', 'Headline', 'Ticker'])
    for i in data.get("feed"):
        test_date = i.get("time_published")
        test_date = test_date[:8]
        newDate = datetime.datetime.strptime(test_date, '%Y%m%d').date()
        row = [newDate, i.get("title"), ticker]
        new_df = pd.DataFrame([row],columns=['Date', 'Headline', 'Ticker'])
        historic_news = pd.concat([historic_news, new_df], axis=0, ignore_index=True)
    return historic_news

#historic_news = getHistoricNewsData('AAPL', '2022-10-10', '2mo')
historic_news = getHistoricNewsData('AAPL', interval = '1mo')
print(historic_news)

endDate is: 2023-02-05
startDate is: 2023-01-06
startDate is: 20230106T0130
endDate is: 20230205T0130
           Date                                           Headline Ticker
0    2023-01-06  Samsung profit slumps by most in decade on wea...   AAPL
1    2023-01-06  Samsung estimates quarterly profit sank to 8-y...   AAPL
2    2023-01-06  Apple's Iconic 1970s Trade Sign, Steve Wozniak...   AAPL
3    2023-01-06  Apple's Mixed Reality Headset 'Behind Schedule...   AAPL
4    2023-01-06  My Top Stock to Buy for 2023  ( and It's Not E...   AAPL
..          ...                                                ...    ...
195  2023-01-12  TSMC Stock Higher On Record Profits, But Muted...   AAPL
196  2023-01-12  Is Meta Platforms Sitting on a $1 Trillion Opp...   AAPL
197  2023-01-12  Should You Invest in the Invesco DWA Technolog...   AAPL
198  2023-01-12                  My Top Tech IPO to Buy in January   AAPL
199  2023-01-12  Taiwan Semiconductor Posts Mixed Fourth-Quarte...   AAPL

[200 rows

# Sentiment Analysis of News data #

In [91]:
def SentimentAnalysisNewsData(parsedNews, printOut = False):
    #Downloading Vader Lexicon for Sentiment Analysis
    nltk.download('vader_lexicon')
    # Initializing Sentiment Analysis
    analyzer = SentimentIntensityAnalyzer()

    #Declaring Column Names
    columns = ['Ticker', 'Date', 'Time', 'Headline']
    #Creating dataframe from news
    news = pd.DataFrame(parsedNews, columns=columns)
    print(news)
    #Getting scores for headlines
    scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

    #Creating Dataframe of Scores
    df_scores = pd.DataFrame(scores)
    #Joining scores to news dataframe
    news = news.join(df_scores, rsuffix='_right')
    #Converting Date column to pd datetime date
    news['Date'] = pd.to_datetime(news.Date).dt.date

    #List of unique tickers
    unique_ticker = news['Ticker'].unique().tolist()
    #Creating dict for news based on ticker
    news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}
    #Initializing List of values
    # og values = []
    values = pd.DataFrame(columns = ['Ticker', 'Date', 'Compound'])
    for ticker in tickers: 
        dataframe = news_dict[ticker]
        dataframe = dataframe.set_index('Ticker')
        #Dropping headlines column since we only need scores now
        dataframe = dataframe.drop(columns = ['Headline'])
        #if printOut:
            #print ('\n')
            #print (dataframe.head())
        
        #mean = round(dataframe['compound'].mean(), 2)
        #Finding compound number for news of every day
        testdf = pd.DataFrame(columns = ['Date', 'Mean Sentiment'])
        testdf = round(dataframe.groupby('Date')['compound'].mean(), 2)
        print(testdf.shape)
        #Adding values to values list
        #og values.append(mean)
    
   
    #print(round(dataframe.groupby('Date')['compound'].mean(), 2))
    #print("VALUES------------")
    #print(values)
        
    #Combining tickers and values into new dataframe
    #df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
    #df = df.set_index('Ticker')
    #df = df.sort_values('date', ascending=False)
    if printOut:
        print("-----------DF")
        #print(df.head())
        #print(df.shape)
    #Returning the dataframe
    return df
    #if printOut:
        #print ('\n')
        #display (df)
    #return df

In [92]:
#print("HISTORIC SENTIMENT")
HistoricSentiment = SentimentAnalysisNewsData(historic_news, True)
#print("\n")
#print("TODAYS SENTIMENT")
#TodaysSentiment = SentimentAnalysisNewsData(parsed_news)

    Ticker        Date  Time  \
0     AAPL  2023-01-06   NaN   
1     AAPL  2023-01-06   NaN   
2     AAPL  2023-01-06   NaN   
3     AAPL  2023-01-06   NaN   
4     AAPL  2023-01-06   NaN   
..     ...         ...   ...   
195   AAPL  2023-01-12   NaN   
196   AAPL  2023-01-12   NaN   
197   AAPL  2023-01-12   NaN   
198   AAPL  2023-01-12   NaN   
199   AAPL  2023-01-12   NaN   

                                              Headline  
0    Samsung profit slumps by most in decade on wea...  
1    Samsung estimates quarterly profit sank to 8-y...  
2    Apple's Iconic 1970s Trade Sign, Steve Wozniak...  
3    Apple's Mixed Reality Headset 'Behind Schedule...  
4    My Top Stock to Buy for 2023  ( and It's Not E...  
..                                                 ...  
195  TSMC Stock Higher On Record Profits, But Muted...  
196  Is Meta Platforms Sitting on a $1 Trillion Opp...  
197  Should You Invest in the Invesco DWA Technolog...  
198                  My Top Tech IPO to Buy i

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ishaan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Creating Dataset #

In [93]:
def createDataset(date_from, int):
    for i in tickers:
        print("Creating dataset for $" +i)
        #Get historic stock data
        historic_stock = getMonthlyStockData(i, interval = int)
        print(historic_stock.head())
        #Get historic news data
        historic_news = getHistoricNewsData(i, interval = int)
        print(historic_news)
        #Use news to get sentiment
        HistoricSentiment = SentimentAnalysisNewsData(historic_news)
        print(HistoricSentiment.shape)
        #Merge as training set
        #Get today's stock data
        #Get today's news data
        #Use news to get sentiment


createDataset('2022-10-10', '2mo')

Creating dataset for $AAPL
Getting stock data for stock $AAPL
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-02-06  152.574997  153.100006  150.779999  151.729996  151.729996   

              Volume  
Date                  
2023-02-06  67990412  
endDate is: 2023-02-05
startDate is: 2022-12-07
startDate is: 20221207T0130
endDate is: 20230205T0130


JSONDecodeError: Expecting value: line 1 column 1 (char 0)